In [1]:
from utils import *
from opt import *
import pickle
import numpy as np

In [2]:
#load the data 
with open('workspace_v1.pkl', 'rb') as file:
    lw = pickle.load(file)

# # Access the variables from the loaded workspace
All_A_hat = lw['All_A_hat']
All_A_hat_bin = lw['All_A_hat_bin']
est_error = lw['est_error']
est_fsc = lw['est_fsc']
est_bias = lw['est_bias']
est_bias_bin = lw['est_bias_bin']
model_fit = lw['model_fit']
true_bias = lw['true_bias']
true_model_fit = lw['true_model_fit']
dnames = lw['dnames']
models = lw['models']
mus2 = lw['mus2']

In [3]:
def print_vector_elements(res):
    # Ensure res is a flat array
    res_flat = res.flatten()
    
    # Create a formatted output string with six spaces after each group of three elements
    output = ""
    for i in range(len(res_flat)):
        if (i + 1) % 3 == 0 and i != len(res_flat) - 1:
            output += f"{res_flat[i]:.4f}      & "
        else:
            output += f"{res_flat[i]:.4f} & "
    
    # Remove the trailing ampersand and space
    output = output.rstrip(" & ")
    
    # Print the formatted string
    np.set_printoptions(suppress=True,precision=4)
    print(output)

#get results for each model 
mdl_idx = 0 
mu2_idx = 0 # for mu = 1 and mu = 1e6
nmodels=len(models)
res = np.zeros((12,nmodels))
for k, dn in enumerate(dnames):
    idx = 3*k 
    for m, model in enumerate(models):
        res[idx,m] = est_error[k,m,mu2_idx]
        res[idx+1,m] = est_bias[k,m,mu2_idx,4]
        res[idx+2,m] = model_fit[k,m,mu2_idx]     



for m, model in enumerate(models):
    print(model)
    print_vector_elements(res[:,m])




GLASSO
0.2661 & 0.3111 & 7.1250      & 0.1995 & 12.6102 & 0.0436      & 0.1121 & 0.9529 & 0.0223      & 0.6477 & 0.4068 & 51.0340
FGLASSO_mu20
0.1417 & 0.4825 & 6.8099      & 0.1895 & 11.8476 & 0.0421      & 0.1119 & 0.8177 & 0.0253      & 0.0505 & 0.1657 & 50.4863
FGLASSO_dp
0.1417 & 0.4824 & 6.8099      & 0.1896 & 10.3317 & 0.0422      & 0.1119 & 0.8177 & 0.0253      & 0.0505 & 0.1657 & 50.4863
FGLASSO_nw
0.1417 & 0.4824 & 6.8099      & 0.1895 & 11.8391 & 0.0421      & 0.1119 & 0.8177 & 0.0253      & 0.0505 & 0.1657 & 50.4863
FGSR_dp
0.4383 & 0.8942 & 7.0651      & 0.4188 & 0.5754 & 1.4283      & 1.0260 & 4.0568 & 0.1724      & 1.0606 & 0.3787 & 51.0690
FGSR_nw
0.4386 & 0.8924 & 7.0652      & 0.4068 & 6.6887 & 1.4221      & 1.0260 & 4.0568 & 0.1724      & 1.1149 & 0.3734 & 51.0663


In [4]:
# Compute the metrics for the rewiting baselines
# Define rewiring 
rw1 = 150
rw2 = 300

nd = len(dnames)
nmetrics = 5

rw_er = np.zeros((nd, 2))
mf = np.zeros((nd, 2))
rw_bias = np.zeros((nd, 2, nmetrics)) # 4 bias metrics
results = np.zeros((12,2))
m = 0

for k, dn in enumerate(dnames):
    dAhat = All_A_hat[dnames[k]]
    dAhat_bin = All_A_hat_bin[dnames[k]]
    A_true, A_true_bin, C_est, C_est_norm, Z, z = load_datasets(dn)
    N = A_true.shape[0]
    #extract each estimated matrix and then rewire links 
    A = dAhat[k,0,0]
    Arw1 = rewire_precmat(A, rewire=rw1, replace=False)
    Arw2 = rewire_precmat(A, rewire=rw2, replace=False)

    #bias,error,modelfit  [0,1,2] [3,4,5]
    rw_er[k,0] = compute_frob_err(Arw1, A_true)
    rw_bias[k,0,:] = compute_all_bias(Arw1,Z)
    rw_er[k,1] = compute_frob_err(Arw2, A_true)
    rw_bias[k,1,:] = compute_all_bias(Arw2,Z)
    mf[k,0] = np.linalg.norm(Arw1 @ C_est_norm - np.eye(N), 'fro')/np.linalg.norm(C_est_norm, 'fro')
    mf[k,1] = np.linalg.norm(Arw2 @ C_est_norm - np.eye(N), 'fro')/np.linalg.norm(C_est_norm, 'fro')

    idx = 3*k        
    results[idx,0] = rw_er[k,0]
    results[idx+1,0] = rw_bias[k,0,4]
    results[idx+2,0] = mf[k,0] 

    results[idx,1] = rw_er[k,1]
    results[idx+1,1] = rw_bias[k,1,4]
    results[idx+2,1] = mf[k,1] 
print(rw_er)
print(rw_bias[:,:,4])

print('GL rw', rw1)
print_vector_elements(results[:,0])
print('GL rw', rw2)
print_vector_elements(results[:,1])

[[0.2679 0.3454]
 [0.2379 0.2921]
 [0.1306 0.1388]
 [0.6494 0.6646]]
[[ 0.331   0.4518]
 [12.6107 12.5812]
 [ 0.9425  0.9171]
 [ 0.4306  0.4072]]
GL rw 150
0.2679 & 0.3310 & 7.0282      & 0.2379 & 12.6107 & 1.8449      & 0.1306 & 0.9425 & 2.1914      & 0.6494 & 0.4306 & 51.0369
GL rw 300
0.3454 & 0.4518 & 7.0327      & 0.2921 & 12.5812 & 1.8302      & 0.1388 & 0.9171 & 2.1814      & 0.6646 & 0.4072 & 51.0378


In [5]:
n_nodes = np.zeros((4,1))
n_edges = np.zeros((4,1))
n_groups = np.zeros((4,1))
n_signals = [28561,1903,943,47127]
sens_Atr= ['Gender','Conference','Year','Gender']
for k, dn in enumerate(dnames):
    A_true, A_true_bin, C_est, C_est_norm, Z, z = load_datasets(dn)
    n_nodes[k] = A_true.shape[0]
    n_edges[k] = np.sum(A_true_bin)/2
    n_groups[k] = Z.shape[0]
    print(dn,': Nodes=',n_nodes[k], ', Edges=', n_edges[k], ', Groups=',n_groups[k], ', Signals=', n_signals[k], ', Sensitive atribute = ', sens_Atr[k])

schoolCs4 : Nodes= [126.] , Edges= [959.] , Groups= [2.] , Signals= 28561 , Sensitive atribute =  Gender
coautorship130 : Nodes= [130.] , Edges= [525.] , Groups= [6.] , Signals= 1903 , Sensitive atribute =  Conference
movielens : Nodes= [200.] , Edges= [665.] , Groups= [2.] , Signals= 943 , Sensitive atribute =  Year
contact311Cs4 : Nodes= [311.] , Edges= [1009.] , Groups= [2.] , Signals= 47127 , Sensitive atribute =  Gender
